In [ ]:
def als_recommender(R, alpha = 40, iterations = 10, lmbda = 0.1, latent = 10):
    
    """
    Derivatives of the cost function, which will be iteratively alternating updated.
    x_u -> user vector
    y_m -> movie vector
    
    x_u = ((Y.T*Y + Y.T*(Cu - I) * Y) + lambda*I)^-1 * (Y.T * Cu * p(u))
    y_m = ((X.T*X + X.T*(Cm - I) * X) + lambda*I)^-1 * (X.T * Cm * p(m))
 
    Args:
        R (pivot_table): user-by-item tensor
 
        alpha (int): The rate in which we'll increase our confidence in a user or movie, 
        according to how many ratings they have
 
        iterations (int): How many times we alternate between fixing and 
        updating our user and movie vectors
 
        lmbda (float): Regularization to prevent overfitting
 
        latent (int): How many latent features we want to compute.
    
    Returns:     
        X (matrix): user vectors of size users-by-latent
        
        Y (matrix): item vectors of size items-by-latent
     """

    #confidence = alpha * rui
    confidence = (R * alpha)

    n_users, n_movies = R.shape

    X = sparse.csr_matrix(np.random.normal(size = (n_users, latent)))
    Y = sparse.csr_matrix(np.random.normal(size = (n_movies, latent)))
    
    #identity matrixes
    X_I = sparse.eye(n_users)
    Y_I = sparse.eye(n_movies)

    #Won't work with the identity of shape (latent, latent) can't add that to shape (n_users, latent)
    I = sparse.eye(latent)
    lI = lmbda * I
    
    #lX = lmbda * X_I
    #lY = lmbda * Y_I
    
    
    #Algorithm for alternating between updating users and movies vectors
    for i in range(iterations):
        
        print ('iteration', i+1, 'of', iterations)
        
        #updated in every interation
        yTy = np.dot(Y.T, Y)
        xTx = np.dot(X.T, X)
        
        #Fix Y (movies) and estimate X (users)
        for user in range(n_users):
            
            print("user:", user)
            
            row = confidence[user,:].toarray()
            print("row:", row.shape)
            
            #Calculate the preference, 0 if the rating doesn't exist, 1 otherwise
            Pu = row.copy()
            Pu[Pu != 0] = 1.0
            print("Pu:", Pu.shape)
            
            #Cu - I part
            CuI = sparse.diags(row, [0])
            Cu = CuI + Y_I
            print("CuI:", CuI.shape, "Cu:", Cu.shape)
            
            #Calculating the derivative formula for the user vectors
            yT_CuI_y = np.dot(np.dot(Y.T, CuI), Y)
            print("yT_CuI_y:", yT_CuI_y.shape)
            yT_Cu_Pu = np.dot(np.dot(Y.T, Cu), Pu) #Transpose?
            print("hey:", yTy.shape, yT_CuI_y.shape, lI.shape, yT_Cu_Pu)
            X[user] = spsolve(yTy + yT_CuI_y + lI, yT_Cu_Pu)
            
        #Fix X (users) and estimate Y (movies)
        for movie in range(n_movies):
            print("movie: ", movie)
            #The idea here is the same as the previous loop but to estimate movies this time
            
            row = confidence[:,movie].T.toarray()
            
            Pm = row.copy()
            Pm[Pm != 0] = 1.0
            
            #Since CmI = Cm - I then Cm = CmI + I
            CmI = sparse.diags(row, [0])
            Cm = CmI + X_I
            
            #Final formula for movie vectors
            xT_CmI_X = np.dot(np.dot(X.T, CmI), X)
            xT_Cm_Pm = np.dot(np.dot(X.T, Cm), Pm.T)
            Y[movie] = spsolve(xTx + xT_CmI_X + lI, xT_Cm_Pm)
            
    return X,Y

In [ ]:
movies = list(np.sort(sampled_ratings.movieId.unique()))
users = list(np.sort(sampled_ratings.userId.unique()))

rows = sampled_ratings.movieId.astype("category").cat.codes.astype(int)
cols = sampled_ratings.userId.astype("category").cat.codes.astype(int)

#similar to pivot_table
pivot_table = sparse.csr_matrix((sampled_ratings.rating, (rows, cols)), shape = (len(movies), len(users)))
pivot_table

In [ ]:
user_vectors, movie_vectors = als_recommender(pivot_table, iterations = 4, latent = 5)
%time
print(user_vectors)
movie_vectors

In [ ]:
#Instead of pivot_table the als from the implicit library expects a sparse matrix
sampled_ratings.movieId = sampled_ratings.movieId.astype("category").cat.codes
sampled_ratings.userId = sampled_ratings.userId.astype("category").cat.codes
movie_user_matrix = sparse.csr_matrix((sampled_ratings.rating.astype(float), (sampled_ratings.movieId, sampled_ratings.userId)))
user_movie_matrix = sparse.csr_matrix((sampled_ratings.rating.astype(float), (sampled_ratings.userId, sampled_ratings.movieId)))

In [ ]:
alpha = 40
confidence = (movie_user_matrix * alpha).astype("double")

#Als model with 10 latent factor, lambda = 0.1 and 10 alternating iterations
als_model = implicit.als.AlternatingLeastSquares(factors = 10, regularization = 0.1, iterations = 10)
als_model.fit(confidence)
%time

In [ ]:
#This recommendation is supposed to be for the vectors returned by als_recommender()
user_vectors = als_model.user_factors
movie_vectors = als_model.item_factors

movie_vec = movie_vectors[10].T

scores = np.dot(movie_vectors, movie_vec)
top_10 = np.argsort(scores)[::-1][:10]


movies = []
movie_scores = []

for i in top_10:
    #get first occorrence of the similar movie and get its title
    movies.append(sampled_ratings.loc[sampled_ratings.movieId == i].title.iloc[0])
    movie_scores.append(scores[i])
    
movies